In [ ]:
import pandas as pd
import biomart 

In [3]:
outp = pd.read_csv("protac_fragments_improved_alt_with_metadata.csv")
uniprotid = outp["Target"].unique().tolist()
len(uniprotid)

59

In [ ]:

# Connect to Ensembl Biomart server
server = biomart.BiomartServer('http://www.ensembl.org/biomart')
mart = server.datasets['hsapiens_gene_ensembl']

# Prepare the query to map UniProt IDs to HUGO gene names
response = mart.search({
    'filters': {
        'uniprotswissprot': uniprotid
    },
    'attributes': ['hgnc_symbol', 'uniprotswissprot']
})

# Parse the response and collect results
mapping = []
for line in response.iter_lines():
    line = line.decode('utf-8')
    fields = line.strip().split('\t')
    if len(fields) >= 2:
        gene_symbol, uniprot_id = fields[0], fields[1]
        mapping.append((uniprot_id, gene_symbol))

# Convert to DataFrame and save to CSV
df = pd.DataFrame(mapping, columns=['UniProt_ID', 'Gene_Name'])
df.to_csv('uniprot_to_gene.csv', index=False)